# Flux Variability Analysis

FreeFlux is capable of solving the canonical [flux balance analysis](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3108565/) (FBA) problem, which is defined as:

\begin{aligned}
\max \quad & {{\bf{c}}^T} \cdot {\bf{v}}\\
s.t. \quad & {\bf{S}} \cdot {\bf{v}} = {\bf{0}}\\
&{{\bf{v}}_{lb}} \le {\bf{v}} \le {{\bf{v}}_{ub}}\\
\end{aligned}

where ${\bf{S}}$ is the stoichiometric matrix, and ${\bf{v}}$ is the flux vector bounded by ${{\bf{v}}_{lb}}$ and ${{\bf{v}}_{ub}}$. The objective function is the linear combination of fluxes, usually the biomass formation.

Here we use the *E. coli* network for example. The model file can be found [here](https://github.com/Chaowu88/freeflux/tree/main/models/ecoli).

In [2]:
from freeflux import Model

MODEL_FILE = 'ecoli_reactions.xlsx' # 'path/to/reactions.xlsx'

model = Model('ecoli')
model.read_from_file(MODEL_FILE)

with model.optimizer() as opt:
    # set bounds for fluxes
    opt.set_flux_bounds('all', bounds = [-100, 100]) 
    opt.set_flux_bounds('glk', bounds = [10, 10])

    # optimization
    opt.prepare()
    res = opt.optimize(objective = {'biom': 1})
print(res)


optimizing [elapsed: 0:00:01]

objective: 1*biom
optimal objective: 1.237
optimal fluxes
pgi: 9.746
pfk: 8.769
fba: 8.769
tpi: 8.769
gapdh: 16.778
eno: 14.625
pk: 10.808
g6pdh: 0.0
gnd: -0.0
rpe: -0.889
rpi: 0.889
tkt1: -0.889
tkt2: 0.668
tkt3: 0.221
tal1: 0.221
tal2: -0.221
edd: 0.0
eda: 0.0
pdh: 7.552
cs: 2.6
can: 2.6
icd: 1.917
kdh: 0.584
suc: -0.0
sdh: 1.267
fum: 1.725
mdh: 2.408
icl: 0.683
ms: 0.683
me1: 0.0
me2: 0.0
ppc: 2.86
ppck: 0.0
ak: 0.0
gdh: 8.075
gs: 0.835
pro: 0.26
arg: 0.348
asp: 2.248
asn: 0.283
ala: 0.604
ser: 1.388
gly1: 0.78
gldc: 0.06
gly2: 0.0
cys: 0.288
lys1: 0.403
lys2: 0.403
thr: 0.639
met: 0.181
val: 0.497
leu: 0.529
ile: 0.341
phe: 0.218
tyr: 0.162
trp: 0.067
his: 0.111
ocm1: 0.181
ocm2: 0.111
atps1: 12.343
atps2: 1.267
nnt: 18.96
atpm: 0.0
glk: 10.0
co2out: 9.651
o2in: 6.805
acout: 0.636
nh4in: 8.611
so4in: 0.288
biom: 1.237


[flux variable analysis](https://pubmed.ncbi.nlm.nih.gov/20920235/) (FVA) is used to find the minimum and maximum of a flux while maintaining objective above some fraction of its optimized value in the original FBA problem. The problem is formulated as: 

\begin{aligned}
\max/min \quad & {\rm{ }}{v_i}\\
s.t. \quad & {\bf{S}} \cdot {\bf{v}} = {\bf{0}}\\
&{{\bf{c}}^T} \cdot {\bf{v}} \ge \gamma  \cdot ob{j_{FBA}}\\
&{{\bf{v}}_{lb}} \le {\bf{v}} \le {{\bf{v}}_{ub}}\\
\end{aligned}

where $\gamma$ controls how close the FBA objective should be to its optiaml value $ob{j_{FBA}}$. 

FreeFlux includes FVA because it can provide a reasonable feasible space for fluxes and initial guesses sampled from it during the least squares optimzation.
Here we 

In [5]:
from freeflux import Model

MODEL_FILE = 'reactions.tsv'

demo = Model('demo')
demo.read_from_file(MODEL_FILE)

with demo.optimizer() as opt:
    # set bounds for fluxes
    opt.set_flux_bounds('all', bounds = [-100, 100]) 
    opt.set_flux_bounds('v7', bounds = [10, 10])
    # FVA
    opt.prepare()
    res = opt.estimate_fluxes_range(objective = {'v3': 1}, gamma = 0)
print(res)


estimating flux ranges [elapsed: 0:00:01]

flux ranges
v1: [10.0, 100.0]
v2: [10.0, 100.0]
v3: [10.0, 10.0]
v4: [0.0, 90.0]
v5: [0.0, 90.0]
v6: [0.0, 90.0]
v7: [10.0, 10.0]
